In [1]:
import sys
sys.path.insert(0, '../')

import os
import pandas as pd 
import numpy as np

from sklearn.preprocessing import StandardScaler
from src.permutation_test import pair_permutation_test

from glob import glob
from pickle import load, dump
from tqdm import tqdm

from src.config import models_features_per
from src.utils import per_error

In [18]:
def normalize_create_training_data(train, test, blank_norm=False, remove_outlier=None):

    # Remove outlier only from the training dataset
    if remove_outlier=='all':
        train = train[train['file'].apply(lambda x: False if (x.split('/')[-1].replace('.txt', '') in ouliners_to_remove) else True)]
        test  = test[test['file'].apply(lambda x: False if (x.split('/')[-1].replace('.txt', '') in ouliners_to_remove) else True)]

    elif remove_outlier=='train_only':
        train = train[train['file'].apply(lambda x: False if (x.split('/')[-1].replace('.txt', '') in ouliners_to_remove) else True)]
        
    train = train.reset_index(drop=True)
    test  = test.reset_index(drop=True)
    
    X_train = train.drop(columns=['file']).copy()
    X_test  = test.drop(columns=['file']).copy()

    columns       = X_train.columns
    
    y_train = train['file'].apply(lambda x: int(x.split('_')[-2].replace('cbz','')))
    y_test  = test['file'].apply(lambda x: int(x.split('_')[-2].replace('cbz','')))

    assert (X_train.index.values == y_train.index.values).all()

    scaler  = StandardScaler()

    if blank_norm: scaler.fit(X_train[y_train==0])
    else: scaler.fit(X_train)

    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=columns)
    X_test  = pd.DataFrame(scaler.transform(X_test),  columns=columns)

    X_train.rename(columns={"PH": 'univariate, max(S)', 'signal_std':'univariate, std(S)', 'signal_mean':'univariate, mean(S)', 'peak area':'univariate, area(S)', \
                        'dS_dV_area':'univariate, area(dS/dV)', 'dS_dV_max_peak':'univariate, max(dS/dV)', 'dS_dV_min_peak':'univariate, min(dS/dV)',\
                    'dS_dV_peak_diff':'univariate, max(dS/dV) - min(dS/dV)', \
                    'peak V':'univariate, V_max(S)', 'dS_dV_max_V':'univariate, V_max(dS/dV)', 'dS_dV_min_V':'univariate, V_min(dS/dV)',\
        }, inplace = True)

    X_test.rename(columns={"PH": 'univariate, max(S)', 'signal_std':'univariate, std(S)', 'signal_mean':'univariate, mean(S)', 'peak area':'univariate, area(S)', \
                        'dS_dV_area':'univariate, area(dS/dV)', 'dS_dV_max_peak':'univariate, max(dS/dV)', 'dS_dV_min_peak':'univariate, min(dS/dV)',\
                    'dS_dV_peak_diff':'univariate, max(dS/dV) - min(dS/dV)', \
                    'peak V':'univariate, V_max(S)', 'dS_dV_max_V':'univariate, V_max(dS/dV)', 'dS_dV_min_V':'univariate, V_min(dS/dV)',\
        }, inplace = True)

    return (X_train, X_test, y_train, y_test), scaler

def load_dataset_train_test_splitted(filename):
    ML1 = pd.read_excel(f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_19_ML1/{filename}.xlsx')
    ML2 = pd.read_excel(f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_22_ML2/{filename}.xlsx')
    ML4 = pd.read_excel(f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML4/{filename}.xlsx')

    return ML1, ML2, ML4

def load_combined_dataset(data_propery, blank_norm=False, remove_outlier=None):
    # Dataset Preparation
    if data_propery=='noisy':
        (ML1_X_train, ML1_X_test, ML1_y_train, ML1_y_test), ML1_scalar = normalize_create_training_data(ML1_noisy_train, ML1_test, blank_norm, remove_outlier)
        (ML2_X_train, ML2_X_test, ML2_y_train, ML2_y_test), ML2_scalar = normalize_create_training_data(ML2_noisy_train, ML2_test, blank_norm, remove_outlier)
        (ML4_X_train, ML4_X_test, ML4_y_train, ML4_y_test), ML4_scalar = normalize_create_training_data(ML4_noisy_train, ML4_test, blank_norm, remove_outlier)
        
    elif data_propery=='augmentation':
        ML1_train_combined = pd.concat([ML1_noisy_train, ML1_train])
        ML2_train_combined = pd.concat([ML2_noisy_train, ML2_train])
        ML4_train_combined = pd.concat([ML4_noisy_train, ML4_train])
        
        (ML1_X_train, ML1_X_test, ML1_y_train, ML1_y_test), ML1_scalar = normalize_create_training_data(ML1_train_combined, ML1_test, blank_norm, remove_outlier)
        (ML2_X_train, ML2_X_test, ML2_y_train, ML2_y_test), ML2_scalar = normalize_create_training_data(ML2_train_combined, ML2_test, blank_norm, remove_outlier)
        (ML4_X_train, ML4_X_test, ML4_y_train, ML4_y_test), ML4_scalar = normalize_create_training_data(ML4_train_combined, ML4_test, blank_norm, remove_outlier)
        
    else:
        (ML1_X_train, ML1_X_test, ML1_y_train, ML1_y_test), ML1_scalar = normalize_create_training_data(ML1_train, ML1_test, blank_norm, remove_outlier)
        (ML2_X_train, ML2_X_test, ML2_y_train, ML2_y_test), ML2_scalar = normalize_create_training_data(ML2_train, ML2_test, blank_norm, remove_outlier)
        (ML4_X_train, ML4_X_test, ML4_y_train, ML4_y_test), ML4_scalar = normalize_create_training_data(ML4_train, ML4_test, blank_norm, remove_outlier)
    
    
    X_train = pd.concat([ML1_X_train, ML2_X_train, ML4_X_train], axis=0)
    y_train = pd.concat([ML1_y_train, ML2_y_train, ML4_y_train], axis=0)
    
    indx_shuffle = np.random.permutation(range(len(X_train)))
    X_train      = X_train.iloc[indx_shuffle]
    y_train      = y_train.iloc[indx_shuffle]
    
    X_test  = pd.concat([ML1_X_test,  ML2_X_test,  ML4_X_test], axis=0)
    y_test  = pd.concat([ML1_y_test,  ML2_y_test,  ML4_y_test], axis=0)

    return X_train, X_test, y_train, y_test

In [19]:
ML1_noisy_train, ML2_noisy_train, ML4_noisy_train = load_dataset_train_test_splitted('extracted_features_noisy_training_dataset')
ML1_train, ML2_train, ML4_train = load_dataset_train_test_splitted('extracted_features_training_dataset')
ML1_test, ML2_test, ML4_test = load_dataset_train_test_splitted('extracted_features_testing_dataset')

# Test if training and testing has common dataset
assert len(set(ML1_train['file'].values.tolist()) & set(ML1_test['file'].values.tolist()))==0

In [21]:
_, X_test, _, y_test = load_combined_dataset(data_propery='noiseless', blank_norm=False, remove_outlier=None)
_, X_test_aug, _, y_test_aug = load_combined_dataset(data_propery='augmentation', blank_norm=False, remove_outlier=None)

In [57]:
model_root_path = os.path.realpath('../models')

model_1_path  = os.path.join(model_root_path, 'Data_augmentation_outlier_remove_None')
model_2_path  = os.path.join(model_root_path, 'Data_augmentation_outlier_remove_train_only')

models_1      =  glob(f'{model_1_path}/*.pickle')
models_2      =  glob(f'{model_2_path}/*.pickle')

common_models = set([os.path.basename(i) for i in models_1]) & set([os.path.basename(i) for i in models_2])

In [58]:
permutation_test_results = pd.DataFrame(columns=['Comp Models', '1st model (% error)', '2nd model (% error)', '% error Diff', 'p value' ])
y_LOD = 0.9117010154341669

compare_same_model_with_aug_nor = False

for model_file_name in tqdm(common_models):
    model_name = model_file_name.replace('.pickle', '')
    
    with open(os.path.join(model_1_path, model_file_name), 'rb') as f:
        model_1 = load(f)

    with open(os.path.join(model_2_path, model_file_name), 'rb') as f:
            model_2 = load(f)

    if not(compare_same_model_with_aug_nor):
        if 'augmentation' in model_1_path:
            model1_pred = model_1.predict(X_test_aug[models_features_per[model_name]])
        else:
            model1_pred = model_1.predict(X_test[models_features_per[model_name]])
            
        if 'augmentation' in model_2_path:
            model2_pred = model_2.predict(X_test_aug[models_features_per[model_name]])
        else:
            model2_pred = model_2.predict(X_test[models_features_per[model_name]])
    
    else:
        model1_pred = model_1.predict(X_test[models_features_per[model_name]])
        model2_pred = model_1.predict(X_test_aug[models_features_per[model_name]])

    per_error_m1 = per_error(y_test, model1_pred, y_LOD)
    per_error_m2 = per_error(y_test, model2_pred, y_LOD)

    observed_r2_score, observed_statistic, p_value, _, _ = pair_permutation_test(model1_pred, model2_pred, y_test, y_LOD)
    temp = pd.DataFrame({'Comp Models':[f'({model_name}) With | Without Outliers'], '1st model (% error)':per_error_m1, '2nd model (% error)':per_error_m2, '% error Diff':[observed_statistic], 'p value':[p_value]})

    permutation_test_results = pd.concat([permutation_test_results, temp], axis=0)

    

  0%|                                                                                                                                                                                      | 0/5 [00:00<?, ?it/s]/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp/T/ipykernel_95068/709678952.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  permutation_test_results = pd.concat([permutation_test_results, temp], axis=0)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:31<00:00,  6.22s/it]


In [33]:
save_loc = '../results/permuation_test/'
os.makedirs(save_loc, exist_ok=True)
permutation_test_results.to_excel(f'{save_loc}/with_without_augmentation_outliers.xlsx', index=False)

In [59]:
permutation_test_results

,Comp Models,1st model (% error),2nd model (% error),% error Diff,p value
0,(KNN) With | Without Outliers,21.870328,19.533164,2.337163,0.0184
0,(GP) With | Without Outliers,25.018578,22.502367,2.516210,0.0516
0,(RF) With | Without Outliers,14.354660,18.773125,-4.418465,0.4765
0,(Linear) With | Without Outliers,46.971488,46.499993,0.471495,0.3683
0,(SVM) With | Without Outliers,22.169956,22.943743,-0.773787,0.5950


In [56]:
permutation_test_results

,Comp Models,1st model (% error),2nd model (% error),% error Diff,p value
0,(KNN) With | Without Outliers,21.870328,18.259551,3.610777,0.1422
0,(GP) With | Without Outliers,25.018578,22.513272,2.505306,0.3795
0,(RF) With | Without Outliers,14.354660,13.463218,0.891442,0.0462
0,(Linear) With | Without Outliers,46.971488,45.395348,1.576140,0.0018
0,(SVM) With | Without Outliers,22.169956,21.523516,0.646439,0.7133


In [31]:
y_LOD = 0.9117010154341669
model_name = 'GP'
with open('../models/Data_augmentation_outlier_remove_None/GP.pickle', 'rb') as f:
    model = load(f)
y_pred = model.predict(X_test[models_features_per[model_name]])

per_error(y_test, y_pred, y_LOD)

37.67141017084106

In [10]:
y_test

0     16
1      0
2      8
3     16
4     16
      ..
41    16
42    16
43     8
44     0
45     8
Name: file, Length: 145, dtype: int64